In [ ]:
from math import log2
import essentia.standard
import numpy as np
from collections import Counter
import os
from time import time
from collections import OrderedDict
from essentia.pytools.spectral import hpcpgram

In [ ]:
melodies_path = 'dataset/chromas_hpcp/vocals_spleeter/'
#melodies_path = 'dataset/chromas_hpcp/vocals_demucs/'
#melodies_path = 'dataset2/chromas_hpcp/vocals_spleeter/'
#melodies_path = 'dataset2/chromas_hpcp/vocals_demucs/'
hummings_path = 'dataset/hummings/'
melodies = os.listdir(melodies_path)
hummings = os.listdir(hummings_path)

In [ ]:
def compute_similarity(_chroma, _chroma_db):
    _crp = essentia.standard.ChromaCrossSimilarity()
    _crp_matrix = _crp(_chroma, _chroma_db)
    _score_matrix, _distance = essentia.standard.CoverSongSimilarity()(_crp_matrix)
    return _distance

In [ ]:
numerador_MRR = 0
numerador_TOP20 = 0
numerador_TOP15 = 0
numerador_TOP10 = 0
numerador_TOP5 = 0
numerador_TOP1 = 0
denominador = 0
tiempo = 0
for hum in hummings:
    denominador += 1
    results = {}
    print("........" + hum + "........")
    print("----------")
    start_time=time()
    loader = essentia.standard.EqloudLoader(filename = hummings_path + hum, sampleRate=44100)
    audio = loader()
    pitch_extractor = essentia.standard.PredominantPitchMelodia(frameSize=2048, hopSize=180)
    pitch_values, _ = pitch_extractor(audio)
    chroma = hpcpgram(pitch_values, sampleRate=44100, hopSize=50)
    for melodie in melodies:
        chroma_db = np.loadtxt(melodies_path + melodie)
        results[melodie] = compute_similarity(chroma_db, chroma)
    ordered_results = OrderedDict({k: v for k, v in sorted(results.items(), key=lambda item: item[1])})
    tiempo += (time()-start_time)
    for i, rank in enumerate(ordered_results.items()):
        if hum.split('.')[0].split('+')[0] == rank[0].split('.')[0]:
            numerador_MRR += (1/(i+1))
            if (i+1) <= 1:
                numerador_TOP1 += 1
            if (i+1) <= 5:
                numerador_TOP5 += 1
            if (i+1) <= 10:
                numerador_TOP10 += 1
            if (i+1) <= 15:
                numerador_TOP15 += 1
            if (i+1) <= 20:
                numerador_TOP20 += 1
        print(i+1, rank[0], rank[1])
        print("----------")
    print("================================")
mrr = numerador_MRR / denominador
top1 = numerador_TOP1 / denominador * 100
top5 = numerador_TOP5 / denominador * 100
top10 = numerador_TOP10 / denominador * 100
top15 = numerador_TOP15 / denominador * 100
top20 = numerador_TOP20 / denominador * 100
tiempo = tiempo / denominador
print('MRR: ', mrr)
print('Top-1: ', top1)
print('Top-5: ', top5)
print('Top-10: ', top10)
print('Top-15: ', top15)
print('Top-20: ', top20)
print('Tiempo: ', tiempo)